## 导入所需要的库

In [1]:
import pandas as pd
import numpy as np
import lightgbm
from pathlib import Path

## 读取文件路径

In [2]:
from data import *

## 加载模型

In [3]:
assert Path(model_data_path).exists(), '无法找到模型文件，请先运行main.ipynb来生成'
model = lightgbm.Booster(model_file=model_data_path)

## 处理测试数据

In [4]:
# 读取测试文件
test_df = pd.read_csv(test_data_path)

In [5]:
#聚合到5分钟粒度
test_df['collect_time'] = pd.to_datetime(test_df['collect_time']).dt.ceil("5min")
#按sum整理一下
test_df = test_df.groupby(['serial_number','collect_time'],as_index=False).agg('sum')
# 分离出要提交的部分
group_min_sn_test = pd.DataFrame(test_df[['serial_number','collect_time']])
# 删除无用数据
test_df.drop(['serial_number', 'collect_time','manufacturer','vendor'], axis=1, inplace=True)

In [6]:
# 预测结果
group_min_sn_test['pti'] = np.argmax(model.predict(test_df).round().astype(np.int), axis=1)

In [7]:
# 去除预测为正常的机器
group_min_sn_test = group_min_sn_test[group_min_sn_test['pti'] != 0]

In [8]:
# 将结果保存为文件
group_min_sn_test.to_csv(output_data_path, header=False, index=False)
print('结果已保存，文件位置：', Path(output_data_path).resolve())

结果已保存，文件位置： C:\Python Projects\K12\result.csv
